In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
# Some important variables that may change over time
team_folder_name = "HSV Fellows Team Folder" # What is the exact name of the shared drive folder
attendance_folder_name = "Budgeting and Programming 2019-2020"

In [ ]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

def main():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
            
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'ncsu_credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    # Create the object that allows us to access drive
    service = build('drive', 'v3', credentials=creds)

    # Call the Drive v3 API
    # This query string specifies that we are looking only for files that fit the mimetype in '', which means folders
    queryString = "mimeType = 'application/vnd.google-apps.folder'"
    # List all of the files that match our query
    results = service.files().list(q=queryString, pageSize=300, fields="nextPageToken, files(name, id)").execute()
    # Now grab those bad boys so we can mess around with them
    items = results.get('files', [])

    # We are going eventually single out the Fellow Team Folder, and we will store its GDrive ID here
    # This could also be found by looking at the long hexadecimal string in the url of the drive folder
    # but we want to find it automatically
    teamFolderID = None
    
    if not items:
        # Hopefully we don't find nothing, but it we do we wanna stop
        print('No files found.')
        return
    else:
        # Otherwise we wanna print out that we found the folder and save the ID
        for item in items:
            if item['name'] == team_folder_name:
                print(u'Found team folder: ({0}, {1})'.format(item['name'], item['id']))
                teamFolderID = item['id']

    if teamFolderID == None:
        # We didn't find the team folder, which is bad
        print('Team folder not found, try checking your team drive folder name and make sure it is exactly correct.')
        return
    
    #for item in items:
        #print(item['name'])

    #return
    
    # This is how many levels of directories we want to descend when searching for the specified folder
    searchDepth = 2
    
    # Define this method so we can use it recursively
    def recursiveListFilesInFolder(rootFolderID, depth, mimeType):
        
        queryString = "'" + rootFolderID + "' in parents"
        folderSearch = service.files().list(q=queryString, pageSize=100, fields="nextPageToken, files(name, id, mimeType)").execute()
        files = folderSearch.get('files', [])

        allFiles = []
        
        for file in files:
            if file['mimeType'] == mimeType:
                allFiles.append(file)
            if depth != 0 and file['mimeType'] == 'application/vnd.google-apps.folder':
                allFiles = allFiles + recursiveListFilesInFolder(file['id'], depth - 1, mimeType)
                
        return allFiles
        
    # Recursively look for folders in the team folder, so we can search for our attendance one
    files = recursiveListFilesInFolder(teamFolderID, searchDepth, 'application/vnd.google-apps.folder')
        
        
    attendanceFolderID = None
    # Look for our attendance folder
    if not files:
        print('No files found.')
    else:
        for file in files:
            if file['name'] == attendance_folder_name:
                print(u'Found attendance folder: ({0}, {1})'.format(file['name'], file['id']))
                attendanceFolderID = file['id']
                
    if attendanceFolderID == None:
        # We didn't find the folder where attendance sheets are kept
        print('Specified attendance folder not found, make sure that the folder name is exactly correct. You can also try increasing the search depth (current={})'.format(searchDepth))
        return
        
    # A depth of -1 means it will search until there are no more directories left ie. infinite depth
    # Let's just hope there aren't any symlinks in this folder :/
    spreadsheets = recursiveListFilesInFolder(attendanceFolderID, -1, 'application/vnd.google-apps.spreadsheet')
    
    #for s in spreadsheets:
    #    print(s)
    
    return s['id']
        
if __name__ == '__main__':
    main()

In [ ]:
page_token = None
while True:
    response = drive_service.files().list(q="mimeType='image/jpeg'",
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name)',
                                          pageToken=page_token).execute()
    for file in response.get('files', []):
        # Process change
        print('Found file: %s (%s)' % (file.get('name'), file.get('id')))
    page_token = response.get('nextPageToken', None)
    if page_token is None:
        break
